In [125]:
import pandas as pd

#Data Loading

In [126]:
df=pd.read_csv('/content/idioms_nlp.csv',encoding='UTF-8')
df

,S.No.,Idioms,Hindi Meaning,English Meaning,Unnamed: 4
0,1.0,A hard nut to crack,xaHkhj leL;k,A difficult problem,NaN
1,2.0,A thorn in the flesh,dke esa ges’kk gksus okyh ck/kk,A constant source of annoyance,NaN
2,3.0,At one’s beck and call,fu;a=.k esa gksuk] vkKk esa gksuk,Under one’s control,NaN
3,4.0,An acid test,xaHkhj ijh{k.k,A critical test,NaN
4,5.0,All eye for an eye,tSls dks rSlk,Tit for tat,NaN
...,...,...,...,...,...
804,990.0,alma matter,ekr`laLFkk,"the school, college or university that somebod...",NaN
805,991.0,go dutch,lk>k djuk,to share the cost of sth with sb,NaN
806,992.0,close fisted,datwl,not willing to spend much money,NaN
807,993.0,high and dry,fu%lgk;,in a difficult situation without money or help,NaN


#Data Preprocessing

In [127]:
df=df.drop(['S.No.','Hindi Meaning','Unnamed: 4'],axis=1)

In [128]:
df.head(20)

,Idioms,English Meaning
0,A hard nut to crack,A difficult problem
1,A thorn in the flesh,A constant source of annoyance
2,At one’s beck and call,Under one’s control
3,An acid test,A critical test
4,All eye for an eye,Tit for tat
5,At once,"At the same time, promptly, instant"
6,At home,Comfortable
7,All in all,Most important
8,Achilles’ heels,A small but fatal weakness
9,Add fuel to fire,To make a matter worse


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809 entries, 0 to 808
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Idioms           809 non-null    object
 1   English Meaning  808 non-null    object
dtypes: object(2)
memory usage: 12.8+ KB


In [130]:
df.isnull().sum()

Idioms             0
English Meaning    1
dtype: int64

In [131]:
df=df[df['Idioms']!='Cave in']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808 entries, 0 to 808
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Idioms           808 non-null    object
 1   English Meaning  808 non-null    object
dtypes: object(2)
memory usage: 18.9+ KB


In [132]:
import re
def remove_brackets(text):
    return re.sub(r'\([^)]*\)', '', text)

In [133]:
df['Idioms']=df['Idioms'].apply(remove_brackets)

In [134]:
df['Idioms']=df['Idioms'].apply(lambda x:'<start> '+ x +' <end>')
df['English Meaning']=df['English Meaning'].apply(lambda x:'<start> '+ x +' <end>')

In [135]:
max_word=max(df['English Meaning'].str.split().str.len())
min_word=min(df['English Meaning'].str.split().str.len())
print("Max words=",int(max_word))
print("Min words=",int(min_word))

Max words= 22
Min words= 3


In [136]:
max_words=max(df['Idioms'].str.split().str.len())
min_words=min(df['Idioms'].str.split().str.len())
print("Max words=",max_words)
print("Min words=",min_words)

Max words= 11
Min words= 3


In [137]:
list_idioms=[]
list_mean=[]
def formlist(text,list):
    list.append(text)


In [ ]:
df['Idioms'].apply(lambda x:formlist(x,list_idioms))
df['English Meaning'].apply(lambda x:formlist(x,list_mean))

In [139]:
print(list_idioms[0],"  ",list_mean[0])

<start> A hard nut to crack <end>    <start> A difficult problem <end>


In [140]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [141]:
def total_words(list):
    tokenizer=Tokenizer(num_words=100)
    tokenizer.fit_on_texts(list)
    return max(tokenizer.word_index.values())

In [142]:
total_idioms_word=total_words(list_idioms)
total_mean_word=total_words(list_mean)
print('total_words_in_idioms=',total_idioms_word)
print('total_words_in_meaning=',total_mean_word)

total_words_in_idioms= 959
total_words_in_meaning= 1268


**Tokenizer_for_idioms_&_their_meaning**

In [143]:
idioms_tokenizer=Tokenizer(total_idioms_word)
idioms_tokenizer.fit_on_texts(list_idioms)

mean_tokenizer=Tokenizer(total_mean_word)
mean_tokenizer.fit_on_texts(list_mean)

In [144]:
def tokenize_pad(list,max_len,num_words):
    tokenizer=Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(list)
    sequence=tokenizer.texts_to_sequences(list)
    padded=pad_sequences(sequence,padding='post',maxlen=max_len)
    return padded

In [145]:
idioms_pad=tokenize_pad(list_idioms,11,total_idioms_word)

In [146]:
mean_pad=tokenize_pad(list_mean,22,total_mean_word)

#** Model Architecture(Seq2Seq_with_teacher_forcing) **

In [153]:
from keras.layers import Input, Embedding, BatchNormalization, LSTM, Dense
from keras.models import Model
from keras.optimizers import Nadam

latent_dim = 100
doc_length = 11
# Encoder model
encoder_input = Input(shape=(doc_length,), name='encoder_input')
x = Embedding(total_idioms_word, latent_dim, name='Idioms_word_embedding')(encoder_input)
x = BatchNormalization(name='Scaling_idioms_vector')(x)
x=LSTM(latent_dim,return_sequences=True,name='Encoder_LSTM1')(x)
_,state_h,state_c = LSTM(latent_dim, return_state=True,return_sequences=True, name='Encoder_LSTM2')(x)
encoder_model = Model(inputs=encoder_input, outputs=[state_h,state_c], name='Encoder_Model')
encoder_out_h,encoder_out_c= encoder_model(encoder_input)

# Decoder model
decoder_input = Input(shape=(None,), name='Decoder_input')
y = Embedding(total_mean_word, latent_dim, name='Mean_word_embedding')(decoder_input)
y = BatchNormalization(name='Scaling_mean_vector')(y)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='Decoder_LSTM')
output_n, _, _ = decoder_lstm(y, initial_state=[encoder_out_h,encoder_out_c])
output_norm = BatchNormalization(name='Decoder_lstm_output_norm')(output_n)

# Dense layer
dense_layer = Dense(total_mean_word, activation='softmax', name='Final_dense_layer')
prob_out = dense_layer(output_norm)

# Sequence_to_Sequence_Model
seq2seq_Model = Model([encoder_input, decoder_input], prob_out)
seq2seq_Model.compile(optimizer=Nadam(learning_rate=0.001), loss='sparse_categorical_crossentropy')
seq2seq_Model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Decoder_input (InputLayer)  [(None, None)]               0         []                            
                                                                                                  
 Mean_word_embedding (Embed  (None, None, 100)            126800    ['Decoder_input[0][0]']       
 ding)                                                                                            
                                                                                                  
 encoder_input (InputLayer)  [(None, 11)]                 0         []                            
                                                                                                  
 Scaling_mean_vector (Batch  (None, None, 100)            400       ['Mean_word_embedding[0]

In [148]:
import numpy as np
np.save('Idioms_vecs.npy',idioms_pad)
np.save('Mean_vecs.npy',mean_pad)


def load_encoder_inputs(encoder_np_vecs='Idioms_vecs.npy'):
    vectorized_body=np.load(encoder_np_vecs)
    encoder_input_data=vectorized_body
    doc_length=encoder_input_data.shape[1]
    print(f'Shape of the encoder input:{encoder_input_data.shape}')
    return encoder_input_data ,doc_length


def load_decoder_inputs(decoder_np_vecs='Mean_vecs.npy'):
    vectorized_title=np.load(decoder_np_vecs)
    decoder_input_data=vectorized_title[:,:-1]
    decoder_target_data=vectorized_title[:,1:]
    print(f'Shape of the decoder input:{decoder_input_data.shape}')
    print(f'Shape of the decoder target :{decoder_target_data.shape}')
    return decoder_input_data,decoder_target_data

decoder_input_data,decoder_target_data=load_decoder_inputs('Mean_vecs.npy')
encoder_input_data,doc_length=load_encoder_inputs('Idioms_vecs.npy')


Shape of the decoder input:(808, 21)
Shape of the decoder target :(808, 21)
Shape of the encoder input:(808, 11)


#Model_training

In [149]:
from keras.callbacks import CSVLogger,ModelCheckpoint
script_name_base='seq2seq'
csv_logger=CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint=ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),save_best_only=True)
batch_size=100
epochs=100
history=seq2seq_Model.fit([encoder_input_data,decoder_input_data], np.expand_dims(decoder_target_data,axis=1),batch_size=batch_size,epochs=epochs,validation_split=0.12,
                          callbacks=[csv_logger,model_checkpoint])

Epoch 1/100
8/8 [==============================] - 13s 425ms/step - loss: 6.8403 - val_loss: 7.0903
Epoch 2/100
1/8 [==>...........................] - ETA: 1s - loss: 6.0581

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 155ms/step - loss: 5.6768 - val_loss: 7.0177
Epoch 3/100
8/8 [==============================] - 2s 224ms/step - loss: 4.8803 - val_loss: 6.9174
Epoch 4/100
8/8 [==============================] - 2s 239ms/step - loss: 4.1303 - val_loss: 6.7878
Epoch 5/100
8/8 [==============================] - 1s 156ms/step - loss: 3.3890 - val_loss: 6.6251
Epoch 6/100
8/8 [==============================] - 1s 154ms/step - loss: 2.7015 - val_loss: 6.4368
Epoch 7/100
8/8 [==============================] - 1s 152ms/step - loss: 2.1528 - val_loss: 6.2548
Epoch 8/100
8/8 [==============================] - 1s 153ms/step - loss: 1.7846 - val_loss: 6.0872
Epoch 9/100
8/8 [==============================] - 1s 167ms/step - loss: 1.5768 - val_loss: 5.9354
Epoch 10/100
8/8 [==============================] - 1s 156ms/step - loss: 1.4509 - val_loss: 5.8221
Epoch 11/100
8/8 [==============================] - 1s 155ms/step - loss: 1.3649 - val_loss: 5.7199
Epoch 12/100
8/8 [==

#Inference_Class_with_beam_search_decoding

In [150]:
from keras.models import load_model
import numpy as np

class IdiomsMeanInference:
    def __init__(self, model_path, idioms_tokenizer, mean_tokenizer, doc_length, beam_width=2):
        self.model = load_model(model_path)
        self.idioms_tokenizer = idioms_tokenizer
        self.mean_tokenizer = mean_tokenizer
        self.doc_length = doc_length
        self.beam_width = beam_width

    def tokenize_idioms(self, idioms_text):
        idioms_text = '<start> ' + idioms_text + ' <end>'
        sequences = self.idioms_tokenizer.texts_to_sequences([idioms_text])
        padded_sequences = pad_sequences(sequences, maxlen=self.doc_length, padding='post')
        return padded_sequences

    def beam_search(self, encoded_idioms, max_length=5):
        start_token = self.mean_tokenizer.word_index['start']
        end_token = self.mean_tokenizer.word_index['end']

        sequences = [[start_token]]
        scores = [0]

        while len(sequences[0]) < max_length:
            all_candidates = []

            for i in range(len(sequences)):
                sequence = sequences[i]
                last_token = sequence[-1]
                if last_token == end_token:
                    continue

                target_seq = np.zeros((1, len(sequence)))
                for t, token in enumerate(sequence):
                    target_seq[0, t] = token

                predictions = self.model.predict([encoded_idioms, target_seq])
                next_token_probs = predictions[0, -1, :]
                best_tokens = np.argsort(next_token_probs)[-self.beam_width:]

                for token in best_tokens:
                    new_sequence = sequence + [token]
                    new_score = scores[i] - np.log(next_token_probs[token])
                    all_candidates.append((new_sequence, new_score))

            if not all_candidates:
                break

            ordered = sorted(all_candidates, key=lambda tup: tup[1])
            sequences = [seq for seq, _ in ordered]
            scores = [score for _, score in ordered]

        best_sequence = sequences[0]
        decoded_sequence = [self.mean_tokenizer.index_word[token] for token in best_sequence]
        return ' '.join(decoded_sequence[1:-1])

    def generate_meaning(self, idioms_text):
        encoded_idioms = self.tokenize_idioms(idioms_text)
        generated_meaning = self.beam_search(encoded_idioms)
        return generated_meaning



#Usage Space

In [152]:

idioms_mean_inference = IdiomsMeanInference('/content/seq2seq.epoch70-val2.52423.hdf5', idioms_tokenizer, mean_tokenizer, doc_length, beam_width=3)
idioms_text = "An iron will"
predicted_meaning = idioms_mean_inference.generate_meaning(idioms_text)
print("Predicted Meaning:", predicted_meaning)


1/1 [==============================] - 0s 28ms/step
Predicted Meaning: strong will power
